In [1]:
import os
from os import listdir
from os.path import isfile, join

import sys

from openpyxl import load_workbook #only in Jupyter

import pandas as pd
import numpy as np
import re

import time
import datetime
import pytz

import string
import random
import math
import matplotlib.pyplot as plt

import zipfile
import io

#for storing files properly
from django.conf import settings

def get_list_of_files(zf):
    fileNames = []
    for i in range(0,len(zf.infolist())):
        if not ("__" in zf.infolist()[i].filename):
            fileNames.append(zf.infolist()[i].filename)
    return fileNames

def write_report(df, sheetName):
    # make random file name
    fileName = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10)) + ".xlsx"
    
    try:
        filePath =  settings.MEDIA_ROOT + fileName
    except:
        filePath =  "/home/devrazdev/" + fileName
    
    # write the output to excel
    writer = pd.ExcelWriter(filePath, engine='openpyxl')
    df.to_excel(writer, sheetName, index=False)

    workbook  = writer.book
    worksheet = writer.sheets[sheetName]

    # automatically scale the columns
    for columnCells in worksheet.columns:
        length = 0
        for cell in columnCells:
            cellValueType = type(cell.value).__name__

            #defaults
            cellText = str(cell.value)
            russianWordsCounter = 0

            if cellValueType == 'unicode':
                cellText = cell.value.encode('utf-8')
                russianWordsCounter = re.search('[а-яА-Я]', cellText)
            else:
                if cellValueType == 'date':
                    cellText = "18/11/1993"

            if russianWordsCounter > 0:
                length = max(length, len(cellText)/2)
            else:
                length = max(length, len(cellText))
            worksheet.column_dimensions[columnCells[0].column].width = length

    writer.save()
    return filePath

/home/devrazdev/.virtualenvs/mysite-virtualenv/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/devrazdev/.virtualenvs/mysite-virtualenv/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
def convert_time(t):
    t_naive = datetime.datetime.fromtimestamp(t)
    timezone = pytz.timezone("UTC")
    t_aware = timezone.localize(t_naive)

    return t_aware

def print_time(t):
    t_aware = convert_time(t)
    return t_aware.astimezone(pytz.timezone('Etc/GMT+7')).strftime("%H:%M:%S")

def convert_to_binary_string(n):
    return ''.join(str(1 & int(n) >> i) for i in range(0, 32)[::-1])

def merge_timestamps(t, t_16):
    unixEpoch = datetime.datetime(1970, 1, 1, 0, 0, 0, tzinfo=pytz.utc)
    javaEpoch = datetime.datetime(1989, 12, 31, 0, 0, 0, tzinfo=pytz.utc)

    epochDelta = (javaEpoch - unixEpoch).total_seconds()

    return int(convert_to_binary_string(t)[:16] + convert_to_binary_string(t_16)[16:], 2) + epochDelta

def process_data_csv(df_in):
    df_hr = df_in[np.isfinite(df_in['monitoring.heart_rate[bpm]'])][["monitoring.timestamp[s]", "monitoring.timestamp_16[s]", "monitoring.heart_rate[bpm]"]].reset_index(drop=True)

    df_hr["timestamp"] = list(map(merge_timestamps, df_hr["monitoring.timestamp[s]"], df_hr["monitoring.timestamp_16[s]"]))

    df_hr["time"] = list(map(print_time, df_hr["timestamp"]))

    df_hr["heart_rate[bpm]"] = df_hr['monitoring.heart_rate[bpm]']

    return df_hr

def add_minutes(df_in):
    df_out = df_in
    df_out["minutes"] = 0

    for i in range(1,len(df_out)):
        df_out.at[i,"minutes"] = math.ceil((convert_time(df_in.at[i,"timestamp"]) - convert_time(df_in.at[0,"timestamp"])).total_seconds() / 60)

    return df_out

In [3]:
inputFolder = os.path.expanduser('~') + "/sample_input/garmin"

# to work with all files in the input folder
inputFiles = [join(inputFolder, f) for f in listdir(inputFolder) if isfile(join(inputFolder, f))]
inputFiles

['/home/devrazdev/sample_input/garmin/23096962014_data.csv',
 '/home/devrazdev/sample_input/garmin/23096988778_data.csv',
 '/home/devrazdev/sample_input/garmin/23109529008_data.csv',
 '/home/devrazdev/sample_input/garmin/23096983288_data.csv',
 '/home/devrazdev/sample_input/garmin/23109530638_data.csv',
 '/home/devrazdev/sample_input/garmin/23099673720_data.csv',
 '/home/devrazdev/sample_input/garmin/23099672116_data.csv',
 '/home/devrazdev/sample_input/garmin/23100089296_data.csv',
 '/home/devrazdev/sample_input/garmin/23082871865_data.csv',
 '/home/devrazdev/sample_input/garmin/23096958794_data.csv',
 '/home/devrazdev/sample_input/garmin/23096946129_data.csv',
 '/home/devrazdev/sample_input/garmin/23096955339_data.csv',
 '/home/devrazdev/sample_input/garmin/23100090964_data.csv',
 '/home/devrazdev/sample_input/garmin/23096953166_data.csv',
 '/home/devrazdev/sample_input/garmin/23096993994_data.csv',
 '/home/devrazdev/sample_input/garmin/23111750966_data.csv',
 '/home/devrazdev/sample

In [7]:
df_out = pd.DataFrame()

for i in range(0, len(inputFiles)):
    #read file
    df_in = pd.read_csv(inputFiles[i], delimiter=",", encoding = 'utf-8')
    try:
        df_tmp = process_data_csv(df_in)
        df_out = pd.concat([df_out, df_tmp])
    except:
        pass

#reshape the output
df_out = add_minutes(df_out.sort_values(by=['timestamp']).reset_index(drop=True))
df_out = df_out[['time', 'minutes', 'heart_rate[bpm]']][df_out["minutes"] > 0][df_out["heart_rate[bpm]"] > 0].drop_duplicates().reset_index(drop=True)

# save the report
#sheetName = convert_time(df_tmp.at[0,"timestamp"]).strftime("%d %b %Y")
#reportPath = write_report(df_out, sheetName)

/home/devrazdev/.virtualenvs/mysite-virtualenv/lib/python3.6/site-packages/IPython/kernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [8]:
plt.plot(list(df_out['minutes']), list(df_out['heart_rate[bpm]']), 'bo', markersize=3)
plt.grid(True)
plt.axis([0, 1440, 0, max(list(df_f['heart_rate[bpm]'])) * 1.1])
plt.show()

NameError: name 'df_f' is not defined